# Anomaly Detection - Coding Exercise

In this coding exercise, you will explore how unsupervised deep learning can be used to detect anomalies. The anomalies are simulated in form of out-of-distribution samples in a set of medical images. Two types of neural networks - an autoencoder and a variational autoencoder - will be implemented to detect these outliers without having access to examples of the anomalies during training.

Authors: Felix Meissen and Martin Menten 

[Chair for AI in Medicine at TU Munich](http://aim-lab.io)

For questions, please contact [felix.meissen@tum.de](mailto:felix.meissen@tum.de)

Munich, 18.11.2021

### Set up directories

In [ ]:
! git init
! git remote add origin https://github.com/martinmenten/anomaly-detection-tutorial.git
! git fetch
! git checkout -t origin/main

### Imports and configuration

In [15]:
from argparse import Namespace
from collections import defaultdict
import random
from warnings import warn

import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_auc_score
import torch
from torch.utils.data import DataLoader

from mednist import (
    MEDNISTDIR,
    download_mednist,
    get_anomal_files,
    get_normal_files,
    MedNISTDataset,
    MedNISTTestDataset
)
from models import Autoencoder, VAE
from utils import plot, plot_anomaly_scores

# Select device to train on
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cpu':
    warn('No GPU found, training will be slow on CPU')

# Config
config = Namespace()
config.batch_size = 128
config.val_split = 0.8
config.test_split = 0.9
config.device = device

# Reproducibility
seed = 0
random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)
torch.backends.cudnn.deterministic = True

# Autoreload modules without restarting the kernel
%load_ext autoreload
%autoreload 2

Training on: cuda
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Download MedNIST and create DataLoader

For this exercise, we will use the [MedNIST](https://medmnist.com) dataset [1,2]. The first cell below downloads parts of the the dataset and prepares training, validation and testing datasets and dataloaders. We define hand x-ray scans as our "normal samples" and create training and validation datasets that consist entirely of such x-rays. The test images, however, are evenly split between hand x-rays and anomal medical images, such as mammographs, chest x-rays as well as head, chest and abdominal computed tomography images. By running the second cell, samples from each dataset are presented.

In [ ]:
# Download if necessary
download_mednist(MEDNISTDIR)

# Get all HeadCT and Hand files
normal_files = get_normal_files(MEDNISTDIR, normal_class='Hand')
anomal_files = get_anomal_files(MEDNISTDIR, normal_class='Hand')
random.shuffle(anomal_files)  # Were sorted by class before
anomal_files = anomal_files[:len(normal_files)]  # Assure number of files are equal

# Create a training / validation / test split
val_split_idx = int(len(normal_files) * config.val_split)
test_split_idx = int(len(normal_files) * config.test_split)

# Take 80% normal images for training
train_files = normal_files[:val_split_idx]

# Take 10% normal images for validation
val_files = normal_files[val_split_idx:test_split_idx]

# Take 10% normal images and an equal amount of anomal images
test_files_1 = normal_files[test_split_idx:]
test_files_2 = anomal_files[test_split_idx:]
test_labels_1 = [0 for _ in range(len(test_files_1))]  # normal files are in-distribution -> 0
test_labels_2 = [1 for _ in range(len(test_files_2))]  # anomal files are out-of-distribution -> 1
test_files = test_files_1 + test_files_2
test_labels = test_labels_1 + test_labels_2

# Create a training dataset with normal files
train_ds = MedNISTDataset(train_files)
trainloader = DataLoader(train_ds, batch_size=config.batch_size, shuffle=True)

# Create a validation dataset with normal files
val_ds = MedNISTDataset(val_files)
valloader = DataLoader(val_ds, batch_size=config.batch_size, shuffle=False)

# Create a test dataset with normal and anomal files
test_ds = MedNISTTestDataset(test_files, test_labels)
testloader = DataLoader(test_ds, batch_size=config.batch_size, shuffle=False)

print('Training dataset size:', len(train_ds))
print('Validation dataset size:', len(val_ds))
print('Test dataset size:', len(test_ds))

In [ ]:
!for f in ls ./data/MedNIST/*; do echo $f; ls $f | wc -l; done

In [ ]:
print('Training images')
plot([img for img in next(iter(trainloader))[:10, 0]])
print('Validation images')
plot([img for img in next(iter(valloader))[:10, 0]])
print('Test images')
imgs, labels = next(iter(testloader))
labels = ["Normal" if label == 0 else "Anomal" for label in labels]
plot([img for img in imgs[:10, 0]], titles=labels[:10])

### Autoencoder

The first anomaly detection strategy is based on a convolutional autoencoder. It consists of an encoder comprised of three convolutional blocks, a bottleneck and a decoder consisting of three convolutional blocks. The next two cells define the training procedure and train the Autoencoder. 

In [ ]:
# Autoencoder training functions

def ae_train_step(ae, x, optimizer, device):
    ae.train()
    optimizer.zero_grad()
    x = x.to(device)
    x_recon = ae(x)
    loss = ae.loss_function(x, x_recon)  # MSE loss
    loss.backward()
    optimizer.step()
    return loss.item()


def ae_val_step(ae, x, device):
    ae.eval()
    x = x.to(device)
    with torch.no_grad():
        x_recon = ae(x)
    return ae.loss_function(x, x_recon).item(), x_recon


def validate_ae(config, ae, valloader):
    losses = []
    for x in valloader:
        loss, x_recon = ae_val_step(ae, x, config.device)
        losses.append(loss)
    return np.mean(losses), x.cpu(), x_recon.cpu()


def train_ae(config, ae, optimizer, trainloader, valloader):
    i_step = 0
    i_epoch = 0
    all_losses = []
    losses = []
    ae.train()
    while True:
        for x in trainloader:
            # Train step
            loss = ae_train_step(ae, x, optimizer, config.device)

            # Store metrics
            losses.append(loss)
            all_losses.append(loss)

            # Log
            if i_step % config.log_frequency == 0:
                print(f'Iteration {i_step} - train loss {np.mean(losses):.4f}')
                losses = []

            # Validate
            if i_step % config.val_frequency == 0:
                val_loss, x_val, x_recon = validate_ae(config, ae, valloader)
                print(f'Iteration {i_step} - val loss {val_loss:.4f}')
                residual = torch.abs(x_val - x_recon)
                plot([x_val[0, 0], x_recon[0, 0], residual[0, 0]],
                     titles=['input', 'reconstruction', 'residual'])

            # Finish
            i_step += 1
            if i_step >= config.num_steps:
                print('Finished training')
                return ae, all_losses
        i_epoch += 1
        print(f'Finished epoch {i_epoch}')

In [ ]:
# Train config
config.lr = 1e-3
config.num_steps = 1000
config.log_frequency = 50
config.val_frequency = 100
config.latent_dim = 128

# Initialize Autoencoder
ae = Autoencoder(latent_dim=config.latent_dim).to(device)
#print(ae)

# Optimizer
optimizer = torch.optim.Adam(ae.parameters(), lr=config.lr)

# Train
print('Start training...')
ae, ae_loss_history = train_ae(config, ae, optimizer, trainloader, valloader)

# Plot loss history
plt.plot(ae_loss_history)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.ylim(0, 0.2)
plt.show()

We test the Autoencoder on the test set defined above. For every sample, we use the mean absolute error of the input and the reconstructed image as anomaly score. We plot some examples with their reconstructions and the error maps and compute the area under the receiver operating characteristics curve (ROC AUC) to see how well our model can distinguish between in- and out-of-distribution samples.

In [ ]:
# Testing

def ae_test_step(ae, x, device):
    ae.eval()
    x = x.to(device)
    with torch.no_grad():
        x_recon = ae(x)
    return x.cpu(), x_recon.cpu(), torch.abs(x - x_recon).cpu()

def test_ae(config, ae, testloader):
    scores = []
    labels = []
    ae.eval()
    for x, y in testloader:
        x, x_recon, residual = ae_test_step(ae, x, config.device)
        anomaly_score = torch.mean(residual, dim=(1, 2, 3))
        scores.extend(anomaly_score.numpy())
        labels.extend(y.numpy())

    # Plot a normal and anomalous test sample
    if len(y == 0) > 0:
        plot([x[y == 0][0, 0], x_recon[y == 0][0, 0], residual[y == 0][0, 0]],
                titles=['input', 'reconstruction', 'residual'])
    if len(y == 1) > 0:
        plot([x[y == 1][0, 0], x_recon[y == 1][0, 0], residual[y == 1][0, 0]],
                titles=['input', 'reconstruction', 'residual'])

    return np.array(scores), np.array(labels)

ae_scores, ae_labels = test_ae(config, ae, testloader)

In [ ]:
# Evaluation
auroc = roc_auc_score(ae_labels, ae_scores)
print(f'ROC AUC: {auroc:.4f}')

plot_anomaly_scores(ae_scores, ae_labels)

# Variational Autoencoder (VAE)

Our VAE has the same architecture as the Autoencoder, but doesn't directly compute the latent vector z in the bottleneck. Instead, it computes the mean and standard deviaton of a Gaussian distribution and samples z from it.
Since it is not possible to backpropagate through a sampling step, we do the "reparametrization trick" as illustrated in the image. We sample from a unit gaussian $N(0,1)$ instead and scale and shift the value by $\sigma$ and $\mu$. This is equivalent to sampling directly from $N(\mu,\sigma)$.

<img src="reparametrization_trick.png" alt="reparametrization trick" width="1300"/>

To train the VAE, we maximize the $ELBO$ (Evidence lower bound) which is a lower bound on the true log likelihood of the data $log\:p(x)$. -> $ELBO \leq log\:p(x)$

$ELBO = likelihood - KL$, where the reconstruction error can be modeled as the negative log likelihood.

So to maximize $ELBO$ (and therefore also $p(x)$), we minimize the reconstruction error and the KL-divergence between $N(\mu,\sigma)$ and $N(0,1)$.

For the derivation of the $ELBO$, have a look at:
[https://fangdahan.medium.com/derivation-of-elbo-in-vae-25ad7991fdf7](https://fangdahan.medium.com/derivation-of-elbo-in-vae-25ad7991fdf7)

In [ ]:
# VAE training functions

def vae_train_step(vae, x, optimizer, device):
    vae.train()
    optimizer.zero_grad()
    x = x.to(device)
    x_recon, mu, logvar = vae(x)
    loss_dict = vae.loss_function(x, x_recon, mu, logvar, kl_weight=2.0)  # VAE loss
    loss = loss_dict['loss']
    loss.backward()
    optimizer.step()
    return loss_dict


def vae_val_step(vae, x, device):
    vae.eval()
    x = x.to(device)
    with torch.no_grad():
        x_recon, mu, logvar = vae(x)
    loss_dict = vae.loss_function(x, x_recon, mu, logvar, kl_weight=2.0)
    return loss_dict, x_recon.cpu()


def validate_vae(config, vae, valloader):
    losses = defaultdict(list)
    for x in valloader:
        loss_dict, x_recon = vae_val_step(vae, x, config.device)
        for k, v in loss_dict.items():
            losses[k].append(v.item())
    losses = {k: np.mean(v) for k, v in losses.items()}
    return losses, x.cpu(), x_recon.cpu()


def train_vae(config, vae, optimizer, trainloader, valloader):
    i_step = 0
    i_epoch = 0
    all_losses = defaultdict(list)
    losses = defaultdict(list)
    vae.train()
    while True:
        for x in trainloader:
            # Train step
            loss_dict = vae_train_step(vae, x, optimizer, config.device)

            # Store metrics
            for k, v in loss_dict.items():
                losses[k].append(v.item())
                all_losses[k].append(v.item())

            # Log
            if i_step % config.log_frequency == 0:
                losses = {k: np.mean(v) for k, v in losses.items()}
                log_msg = f'Iteration {i_step}'
                for k, v in losses.items():
                    log_msg += f' - {k}: {v:.4f}'
                print(log_msg)
                losses = defaultdict(list)

            # Validate
            if i_step % config.val_frequency == 0:
                val_losses, x_val, x_recon = validate_vae(config, vae, valloader)
                log_msg = f'Iteration {i_step}'
                for k, v in val_losses.items():
                    log_msg += f' - val {k}: {v:.4f}'
                print(log_msg)
                residual = torch.abs(x_val - x_recon)
                plot([x_val[0, 0], x_recon[0, 0], residual[0, 0]],
                     titles=['input', 'reconstruction', 'residual'])

            # Finish
            i_step += 1
            if i_step >= config.num_steps:
                print('Finished training')
                return vae, all_losses
        i_epoch += 1
        print(f'Finished epoch {i_epoch}')

In [ ]:
# Initialize Variational Autoencoder
vae = VAE(latent_dim=config.latent_dim).to(device)

# Optimizer
optimizer = torch.optim.Adam(vae.parameters(), lr=config.lr)

# Train
print('Start training...')
vae, vae_loss_history = train_vae(config, vae, optimizer, trainloader, valloader)

# Plot loss history
for k, v in vae_loss_history.items():
    plt.plot(v, label=k)
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.ylim(0, 0.2)
plt.show()

In [ ]:
# Testing VAE

def vae_test_step(vae, x, device):
    vae.eval()
    x = x.to(device)
    with torch.no_grad():
        anomaly_scores = vae.anomaly_score(x)
    return anomaly_scores.cpu()

def test_vae(config, vae, testloader):
    scores = []
    labels = []
    vae.eval()
    for x, y in testloader:
        anomaly_scores = vae_test_step(vae, x, config.device)
        scores.extend(anomaly_scores.numpy())
        labels.extend(y.numpy())

    return np.array(scores), np.array(labels)

vae_scores, vae_labels = test_vae(config, vae, testloader)

In [ ]:
# Evaluation
auroc = roc_auc_score(vae_labels, vae_scores)
print(f'ROC AUC: {auroc:.4f}')

plot_anomaly_scores(vae_scores, vae_labels)

### Conclusion

With self-supervised methods we can perform tasks without the need for labels. Anomaly detection can even segment anomalous regions like tumors in an image. However, this is still ongoing research.
In our experiments, the reconstruction-based method performed better, than the likelihood-based method. This is not always the case, using the reconstruction error as anomaly scoring function can have severe drawbacks in other cases.

### References
[1] Jiancheng Yang, Rui Shi, Donglai Wei, Zequan Liu, Lin Zhao, Bilian Ke, Hanspeter Pfister, Bingbing Ni. "MedMNIST v2: A Large-Scale Lightweight Benchmark for 2D and 3D Biomedical Image Classification". arXiv preprint arXiv:2110.14795, 2021.

[2] Jiancheng Yang, Rui Shi, Bingbing Ni. "MedMNIST Classification Decathlon: A Lightweight AutoML Benchmark for Medical Image Analysis". IEEE 18th International Symposium on Biomedical Imaging (ISBI), 2021.